##### 0.1. Importing all the needed libraries

In [1]:
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split as tts
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch import optim
import numpy as np
import pandas as pd
import torch
import optuna
import os

##### 0.2. Setting some global settins

In [2]:
%matplotlib ipympl

##### 0.3. Importing the data file


##### 1.1. Building the Neural Network

In [3]:
def clp_network(trial=False, n_l=-1, units=[]):
    custom_network = not trial and len(units) == n_l
    if custom_network: n_layers = n_l
    else: n_layers = trial.suggest_int("n_layers", 2, 5)
    layers = []

    in_features = 6
    
    for i in range(n_layers):
        
        if custom_network: out_features = units[i]
        else: out_features = trial.suggest_int("n_units_l{}".format(i), 4, 10)
        
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.LeakyReLU())

        in_features = out_features
        
    layers.append(nn.Linear(in_features, 2))
    layers.append(nn.LeakyReLU())
    
    return nn.Sequential(*layers)

##### 1.2. Making a Dataset class

In [4]:
class clp_dataset(Dataset):

    def __init__(self, df):
        self.labels = [label for label in df['loyalty']]
        self.features = df.drop(columns=['loyalty'], axis=1).values.tolist()

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        return np.array(self.labels[idx])

    def get_batch_features(self, idx):
        return np.array(self.features[idx])

    def __getitem__(self, idx):
        batch_features = self.get_batch_features(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_features, batch_y

##### 1.3. Training and evaluation of the model

In [5]:
def train_and_evaluate(param, model, trial=False):

    # extracting parameters
    if 'n_epochs' in param: n_epochs = param['n_epochs']
    else: n_epochs = 50 # DEFAULT VALUE

    if 'batch_size' in param: batch_size = param['batch_size']
    else: batch_size = 32 # DEFAULT VALUE

    # if 'units' in param: units = param['units']
    # else: units = []

    # if 'n_layers' in param: n_layers = param['n_layers']
    # else: n_layers = -1
    # # checking if we're using a custom model
    # custom_model = not trial and len(units) == n_layers
    
    # importing data from file
    data_path = os.path.join(os.getcwd(), 'Data\\processed_customer_data.csv')
    data = pd.read_csv(data_path)
    data.drop(columns='ID', inplace=True)
    
    train_data, val_data = tts(data, test_size = 0.2)
    train, val = clp_dataset(train_data), clp_dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=batch_size)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = getattr(optim, param['optimizer'])(model.parameters(), lr= param['learning_rate'])

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(n_epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in train_dataloader:

                train_label = train_label.to(device)
                train_input = train_input.to(device)

                output = model(train_input.float())
                
                batch_loss = criterion(output, train_label.long())
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    val_input = val_input.to(device)

                    output = model(val_input.float())

                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            accuracy = total_acc_val/len(val_data)
            
            # Add prune mechanism
            if trial:
                trial.report(accuracy, epoch_num)

                if trial.should_prune():
                    raise optuna.exceptions.TrialPruned()

    return accuracy, model

##### 1.4. Making the objective function

In [6]:
def objective(trial, n_epochs, batch_size):
     ### ADD OTHER PARAMS
     params = {
              'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-1),
              'optimizer': trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"]),
              'n_epochs': n_epochs,
              'batch_size': batch_size
              }
     model = clp_network(trial)
     accuracy, _ = train_and_evaluate(params, model, trial)
     return accuracy

In [7]:
bests = []

# for e in range(30,151,30): ########################################################
for e in range(10,11,100):
    # for b in [4, 8, 16, 32, 64]: ###################################################
    for b in [64]:
        study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.MedianPruner())
        # study.optimize(lambda trial: objective(trial, e, b), n_trials=20) ############################################
        study.optimize(lambda trial: objective(trial, e, b), n_trials=2)
        best_params = study.best_trial
        param_dict = best_params.params
        print(f"Study:    n_epoch = {e} batch_size = {b}")
        print(f"accuracy: {best_params.value}")

        params_dict = dict()
        params_dict['params'] = param_dict
        params_dict['value'] = best_params.value
        params_dict['n_epoch'] = e
        params_dict['batch_size'] = b
        bests.append(params_dict)

for i in range(len(bests)):
    param_dict = bests[i]['params']
    print("Study " + str(i) + ":    n_epoch = " + str(bests[i]['n_epoch']) + " batch_size = " + str(bests[i]['batch_size']))
    print('accuracy: ' + str(bests[i]['value']))
    for k, v in param_dict.items():
        print(k, end=' ')
        print(v)

[I 2022-08-30 12:45:05,988] A new study created in memory with name: no-name-025e9ac5-0011-4937-a1ac-054c8e86416d
[I 2022-08-30 12:45:12,156] Trial 0 finished with value: 0.7214285714285714 and parameters: {'learning_rate': 0.00010637794544522072, 'optimizer': 'RMSprop', 'n_layers': 2, 'n_units_l0': 7, 'n_units_l1': 4}. Best is trial 0 with value: 0.7214285714285714.
[I 2022-08-30 12:45:12,703] Trial 1 finished with value: 0.7571428571428571 and parameters: {'learning_rate': 4.395858286933057e-05, 'optimizer': 'SGD', 'n_layers': 4, 'n_units_l0': 5, 'n_units_l1': 8, 'n_units_l2': 5, 'n_units_l3': 7}. Best is trial 1 with value: 0.7571428571428571.


Study:    n_epoch = 10 batch_size = 64
accuracy: 0.7571428571428571
Study 0:    n_epoch = 10 batch_size = 64
accuracy: 0.7571428571428571
learning_rate 4.395858286933057e-05
optimizer SGD
n_layers 4
n_units_l0 5
n_units_l1 8
n_units_l2 5
n_units_l3 7


In [8]:
# def clp_network(trial=False, n_l=-1, n_u=[]):
# params_dict = dict()
# params_dict['params'] = param_dict
# params_dict['value'] = best_params.value
# params_dict['n_epoch'] = e
# params_dict['batch_size'] = b
# bests.append(params_dict)

max_index = 0
for i in range(1, len(bests)):
    if bests[i]['value'] > bests[max_index]['value']: max_index = i

best_case = bests[max_index]
best_accuracy = best_case['value']
best_n_epochs = best_case['n_epoch']
best_batch_size = best_case['batch_size']

best_params = best_case['params']

best_lr = best_params['learning_rate']
best_optimizer = best_params['optimizer']
best_n_layers = best_params['n_layers']

best_n_units = []
for i in range(best_n_layers):
    best_n_units.append(best_params["n_units_l{}".format(i)])

